In [2]:
# OS library
import sys
sys.path.append('../')

import glob
import pandas as pd
import numpy as np

from urllib.request import urlopen
import json

# Magic to reload notebook
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = 100

# Covid Saúde

Painel de casos de doença pelo coronavírus 2019 (COVID-19) no Brasil pelo Ministério da Saúde ([website](https://covid.saude.gov.br/))

In [ ]:
df_brasil = pd.read_csv('../data/app/results/df_brasil.csv')
df_regional = pd.read_csv('../data/app/results/df_regional.csv')
df_uf = pd.read_csv('../data/app/results/df_uf.csv')
df_regional_saude = pd.read_csv('../data/app/results/df_regional_saude.csv')
df_mun = pd.read_csv('../data/app/results/df_mun.csv')

In [ ]:
df_covid_saude = pd.concat([df_brasil, df_regional, df_uf, df_regional_saude, df_mun])
del df_covid_saude['index']

In [ ]:
df_covid_saude.loc[df_covid_saude['obitosNovos']<0, 'obitosNovos'] = 0
df_covid_saude.loc[df_covid_saude['obitosAcumulado']<0, 'obitosAcumulado'] = 0

In [ ]:
# df_covid_saude = pd.read_parquet("../data/processed/covid_saude.parquet")
df_acoes = pd.read_csv('../data/raw/boletim_covid/acoes.csv', sep='\t', encoding='latin-1')
df_referencias = pd.read_csv('../data/raw/boletim_covid/referencias.csv', sep='\t', encoding='latin-1')
df_death_predictions = pd.read_csv('../data/app/prediction_two_models.csv', encoding="ISO-8859-1")
df_predictions_waves = pd.read_csv('../data/app/ajusteSPondas.csv', encoding='latin-1')

In [ ]:
# df_covid_saude = pd.read_parquet("../data/processed/covid_saude.parquet")
df_acoes = pd.read_csv('../data/raw/boletim_covid/acoes.csv', sep='\t', encoding='latin-1')
df_referencias = pd.read_csv('../data/raw/boletim_covid/referencias.csv', sep='\t', encoding='latin-1')
df_death_predictions = pd.read_csv('../data/app/prediction_two_models.csv', encoding="ISO-8859-1")

In [ ]:
df_death_predictions['data'] = pd.to_datetime(df_death_predictions['data'])

df_covid_saude_mun = df_covid_saude.loc[df_covid_saude['municipio'].notna()]

df_covid_saude_mun = df_covid_saude_mun[[
    'regiao', 'estado', 'municipio', 'codmun', 'nomeRegiaoSaude',
    'codRegiaoSaude', 'data', 'obitosNovos', 'obitosAcumulado'
]]

df_covid_saude_mun['data'] = pd.to_datetime(df_covid_saude_mun['data'])

df_depara_wn_day = pd.concat([df_covid_saude_mun[['data']].drop_duplicates(), df_death_predictions[['data']].drop_duplicates()])
df_covid_saude_mun['data'] = pd.to_datetime(df_covid_saude_mun['data'])
df_depara_wn_day['weekday'] = df_depara_wn_day['data'].dt.weekday
df_depara_wn_day['week_number'] = 0
df_depara_wn_day.loc[df_depara_wn_day['weekday'] == 0, 'week_number'] = 1
df_depara_wn_day['groupby_col'] = 1
df_depara_wn_day['week_number'] = df_depara_wn_day.groupby(
    'groupby_col')['week_number'].cumsum()
df_depara_wn_day['week_number_day'] = df_depara_wn_day['week_number'] + df_depara_wn_day['weekday']/7 # used to plot
df_death_predictions = df_death_predictions.merge(df_depara_wn_day[['data', 'week_number_day']], how='left', on='data')

df_covid_saude_mun = df_covid_saude_mun.merge(
    df_depara_wn_day[['data', 'week_number']], how='left', on=['data'])

df_covid_saude_mun.sort_values(by=['municipio', 'data'],
                               ascending=[True, True],
                               inplace=True)
df_covid_saude_br = df_covid_saude_mun[['week_number',
                                        'data']].drop_duplicates(
                                            subset=['week_number'],
                                            keep='first')
df_covid_saude_reg = df_covid_saude_mun[['regiao', 'week_number',
                                         'data']].drop_duplicates(
                                             subset=['regiao', 'week_number'],
                                             keep='first')
df_covid_saude_state = df_covid_saude_mun[[
    'regiao', 'estado', 'week_number', 'data'
]].drop_duplicates(subset=['estado', 'week_number'], keep='first')
df_covid_saude_regsau = df_covid_saude_mun[[
    'estado', 'nomeRegiaoSaude', 'codRegiaoSaude', 'week_number', 'data'
]].drop_duplicates(subset=['nomeRegiaoSaude', 'codRegiaoSaude', 'week_number'],
                   keep='first')
df_covid_saude_city = df_covid_saude_mun[[
    'nomeRegiaoSaude', 'municipio', 'codmun', 'week_number', 'data'
]].drop_duplicates(subset=['municipio', 'codmun', 'week_number'], keep='first')

df_weekly_deaths_level_up_br = df_covid_saude_mun.groupby(
    ['week_number'])['obitosNovos'].sum().reset_index(name='new_deaths_week')
df_weekly_deaths_level_up_reg = df_covid_saude_mun.groupby(
    ['week_number'])['obitosNovos'].sum().reset_index(name='new_deaths_week')
df_weekly_deaths_level_up_state = df_covid_saude_mun.groupby(
    ['regiao', 'week_number'])['obitosNovos'].sum().reset_index(name='new_deaths_week')
df_weekly_deaths_level_up_regsau = df_covid_saude_mun.groupby(
    ['estado', 'week_number'])['obitosNovos'].sum().reset_index(name='new_deaths_week')
df_weekly_deaths_level_up_city = df_covid_saude_mun.groupby(
    ['nomeRegiaoSaude', 'week_number'])['obitosNovos'].sum().reset_index(name='new_deaths_week')

df_weekly_deaths_br = df_covid_saude_mun.groupby([
    'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')
df_weekly_deaths_region = df_covid_saude_mun.groupby([
    'regiao', 'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')
df_weekly_deaths_state = df_covid_saude_mun.groupby([
    'regiao', 'estado', 'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')
df_weekly_deaths_regsau = df_covid_saude_mun.groupby([
    'estado', 'nomeRegiaoSaude', 'codRegiaoSaude', 'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')
df_weekly_deaths_city = df_covid_saude_mun.groupby([
    'nomeRegiaoSaude', 'municipio', 'codmun', 'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')

In [ ]:
df_covid_saude_br = df_covid_saude_br.merge(df_weekly_deaths_br, how='left', on=['week_number']) \
    .merge(df_weekly_deaths_level_up_br, how='left', on=['week_number'])
df_covid_saude_reg = df_covid_saude_reg.merge(df_weekly_deaths_region, how='left', on=['regiao', 'week_number']) \
    .merge(df_weekly_deaths_level_up_reg, how='left', on=['week_number'])
df_covid_saude_state = df_covid_saude_state.merge(df_weekly_deaths_state, how='left', on=['regiao', 'estado', 'week_number']) \
    .merge(df_weekly_deaths_level_up_state, how='left', on=['regiao', 'week_number'])
df_covid_saude_regsau = df_covid_saude_regsau.merge(df_weekly_deaths_regsau, how='left', on=['estado', 'nomeRegiaoSaude', 'codRegiaoSaude', 'week_number']) \
    .merge(df_weekly_deaths_level_up_regsau, how='left', on=['estado', 'week_number'])
df_covid_saude_city = df_covid_saude_city.merge(df_weekly_deaths_city, how='left', on=['nomeRegiaoSaude', 'municipio', 'codmun', 'week_number']) \
    .merge(df_weekly_deaths_level_up_city, how='left', on=['nomeRegiaoSaude', 'week_number'])

df_covid_saude_br['percentage_deaths'] = df_covid_saude_br[
    'new_deaths_week_division'] / df_covid_saude_br['new_deaths_week']
df_covid_saude_reg['percentage_deaths'] = df_covid_saude_reg[
    'new_deaths_week_division'] / df_covid_saude_reg['new_deaths_week']
df_covid_saude_state['percentage_deaths'] = df_covid_saude_state[
    'new_deaths_week_division'] / df_covid_saude_state['new_deaths_week']
df_covid_saude_regsau['percentage_deaths'] = df_covid_saude_regsau[
    'new_deaths_week_division'] / df_covid_saude_regsau['new_deaths_week']
df_covid_saude_city['percentage_deaths'] = df_covid_saude_city[
    'new_deaths_week_division'] / df_covid_saude_city['new_deaths_week']

df_covid_saude_br.fillna(0, inplace=True)
df_covid_saude_reg.fillna(0, inplace=True)
df_covid_saude_state.fillna(0, inplace=True)
df_covid_saude_regsau.fillna(0, inplace=True)
df_covid_saude_city.fillna(0, inplace=True)

df_covid_saude_grouped = pd.concat([
    df_covid_saude_city, df_covid_saude_regsau, df_covid_saude_state,
    df_covid_saude_reg, df_covid_saude_br
])

##################################################################
# Tratamento de dados das noticias
##################################################################

df_acoes['texto'] = df_acoes['texto'].str.replace('\x93', '"')\
                                     .str.replace('\x94', '"')\
                                     .str.replace('\x91', "'")\
                                     .str.replace('\x92', "'")\
                                     .str.replace('\x85', '...')

df_acoes['data'] = pd.to_datetime(df_acoes['data'], format='%d/%m/%y')

df_acoes['texto'] = df_acoes['texto'].str.wrap(30)
df_acoes['texto'] = df_acoes['texto'].apply(lambda x: x.replace('\n', '<br>'))
df_acoes = df_acoes.merge(df_depara_wn_day, how='left', on='data')

df_acoes['noticia'] = '<b>Data: </b>' + df_acoes['data'].dt.strftime(
    '%d/%m/%Y') + '<br><b>Tipo: </b>' + df_acoes['tipo'].astype(
        str) + '<br><b>Notícia: </b>' + df_acoes['texto']

df_noticias = df_acoes.groupby([
    'week_number'
])['noticia'].apply(lambda x: '<br><br>'.join(x)).reset_index()

# df_noticias['noticia'] = '<div style="height:120px;width:120px;border:1px solid #ccc;font:16px/26px Georgia, Garamond, Serif;overflow:auto;">' + df_noticias['noticia'] + '</div>'
# df_noticias['noticia'] = '<p>' + df_noticias['noticia'] + '</p>'
df_covid_saude_grouped = df_covid_saude_grouped.merge(df_noticias,
                                                      how='left',
                                                      on='week_number')

df_covid_saude_grouped['noticia'] = df_covid_saude_grouped['noticia'].where(pd.notnull(df_covid_saude_grouped['noticia']), None)

In [ ]:
# TREAT COVID DATA TO GET "TIPO" VARIABLE

In [ ]:
import pandas as pd
df_acoes = pd.read_csv('C:/Users/mscamargo/Desktop/estudos/my_proj/covid19_previsoes_municipios/data/raw/boletim_covid/acoes.csv', sep='\t', encoding='latin-1')
df_referencias = pd.read_csv('C:/Users/mscamargo/Desktop/estudos/my_proj/covid19_previsoes_municipios/data/raw/boletim_covid/referencias.csv', sep='\t', encoding='latin-1')


In [ ]:
df_acoes

In [ ]:
df_acoes['data'] = pd.to_datetime(df_acoes['data'])
df_acoes['weekday'] = df_acoes['data'].dt.weekday
df_acoes['week_number'] = 0
df_acoes.loc[df_acoes['weekday'] == 0, 'week_number'] = 1
df_acoes['groupby_col'] = 1
df_acoes['week_number'] = df_acoes.groupby(
    'groupby_col')['week_number'].cumsum()
df_acoes['week_number_day'] = df_acoes['week_number'] + df_acoes['weekday']/7 # used to plot

In [ ]:
df_acoes

In [ ]:
##################################################################
# Tratamento de dados das noticias
##################################################################

df_acoes['texto'] = df_acoes['texto'].str.replace('\x93', '"')\
                                     .str.replace('\x94', '"')\
                                     .str.replace('\x91', "'")\
                                     .str.replace('\x92', "'")\
                                     .str.replace('\x85', '...')

df_acoes['data'] = pd.to_datetime(df_acoes['data'], format='%d/%m/%y')

df_acoes['texto'] = df_acoes['texto'].str.wrap(30)
df_acoes['texto'] = df_acoes['texto'].apply(lambda x: x.replace('\n', '<br>'))
#df_acoes = df_acoes.merge(df_depara_wn_day, how='left', on='data')

df_acoes['noticia'] = '<b>Data: </b>' + df_acoes['data'].dt.strftime(
    '%d/%m/%Y') + '<br><b>Tipo: </b>' + df_acoes['tipo'].astype(
        str) + '<br><b>Notícia: </b>' + df_acoes['texto']

df_noticias = df_acoes.groupby([
    'data', 'tipo'
])[['noticia']].apply(lambda x: '<br><br>'.join(x)).reset_index()

# df_noticias['noticia'] = '<div style="height:120px;width:120px;border:1px solid #ccc;font:16px/26px Georgia, Garamond, Serif;overflow:auto;">' + df_noticias['noticia'] + '</div>'
# df_noticias['noticia'] = '<p>' + df_noticias['noticia'] + '</p>'


In [ ]:
df = df_noticias[['data', 'tipo']].drop_duplicates()

In [ ]:
df['tipo'].unique()

In [ ]:
import numpy as np
df['tipo'] = np.where(df['tipo'] == '0 Propaganda', 'Propaganda', df['tipo'])
df['tipo'] = np.where(df['tipo'] == '1 Propaganda', 'Propaganda', df['tipo'])
df['tipo'] = np.where(df['tipo'] == 'Atos de governo', 'Atos de Governo', df['tipo'])

In [ ]:
df['tipo_at'] = df.groupby(['data'])['tipo'].transform(lambda x: ','.join(x))
df_finally_not = df[['data','tipo_at']].drop_duplicates()

In [ ]:
df_weekly_deaths = pd.read_parquet('C:/Users/mscamargo/Desktop/estudos/my_proj/covid19_previsoes_municipios/data/app/covid_saude_obito_grouped.parquet')


In [ ]:
ok['tipo_at'].unique()

In [ ]:
ok = df_weekly_deaths.merge(df_finally_not, on = 'week_number', how = 'left')
ok.to_parquet('C:/Users/mscamargo/Desktop/estudos/my_proj/covid19_previsoes_municipios/data/app/covid_saude_obito_grouped_new.parquet', index=False)

In [ ]:
df_covid_saude_grouped.to_parquet('../data/app/covid_saude_obito_grouped_v3.parquet', index=False)

In [ ]:
df_death_predictions.to_parquet('../data/app/death_predictions.parquet', index=False)

In [ ]:
df_teste = pd.read_parquet('../data/app/est_cidade.parquet')

In [ ]:
df_teste.drop_duplicates().dropna().to_parquet('../data/app/est_cidade.parquet', index=False)

In [ ]:
df_predictions_waves = pd.read_csv('../data/app/ajusteSPondasTeste2.csv', encoding='latin-1')

In [ ]:
del df_predictions_waves['Unnamed: 0']

In [ ]:
df_predictions_waves = df_predictions_waves[(df_predictions_waves['obitosPreditos'].notna())]

In [ ]:
df_predictions_waves.rename(columns={'date':'data'}, inplace=True)

In [ ]:
df_predictions_waves.to_parquet('../data/app/ajusteSPondasTeste2.parquet', index=False)

In [ ]:
df_predictions_waves.loc[df_predictions_waves['obitosAcumPreditos.lower'].notna(), 'onda'].unique()

In [ ]:
df_covid_saude_mun.to_parquet('../data/app/covid_saude_obitos_diarios_v3.parquet', index=False)

# To deploy

In [6]:
df_covid_saude = pd.read_parquet("../data/processed/covid_saude.parquet")

In [11]:
list_cities_sp = list(df_covid_saude.loc[(df_covid_saude['codmun'].notna())&(df_covid_saude['estado']=='SP'), 'codmun'].unique())
list_reg_saude_sp = list(df_covid_saude.loc[(df_covid_saude['codRegiaoSaude'].notna())&(df_covid_saude['estado']=='SP'), 'codRegiaoSaude'].unique())

In [3]:
df_weekly_deaths = pd.read_parquet('../data/app/covid_saude_obito_grouped.parquet')
df_daily_deaths = pd.read_parquet('../data/app/covid_saude_obitos_diarios.parquet')
df_depara_levels = pd.read_parquet('../data/app/depara_levels.parquet')
df_regional_clusters = pd.read_parquet('../data/app/clusters.parquet')
df_death_predictions = pd.read_parquet('../data/app/death_predictions.parquet')
df_predictions_waves = pd.read_parquet('../data/app/ajuste_ondas.parquet')

In [25]:
df_weekly_deaths = df_weekly_deaths.loc[(df_weekly_deaths['codmun'].isin(list_cities_sp))
                     |(df_weekly_deaths['codRegiaoSaude'].isin(list_reg_saude_sp))
                    |((df_weekly_deaths['codmun'].isna())
                    &(df_weekly_deaths['codRegiaoSaude'].isna()))]

In [28]:
df_daily_deaths = df_daily_deaths.loc[(df_daily_deaths['codmun'].isin(list_cities_sp))
                     |(df_daily_deaths['codRegiaoSaude'].isin(list_reg_saude_sp))
                    |((df_daily_deaths['codmun'].isna())
                    &(df_daily_deaths['codRegiaoSaude'].isna()))]

In [34]:
df_death_predictions = df_death_predictions.loc[df_death_predictions['codmun'].isin(list_cities_sp)]

In [35]:
df_predictions_waves

,estado,municipio,codmun,data,obitosPreditos,obitosAcumPreditos,onda,obitosPreditos.lower,obitosPreditos.upper,obitosAcumPreditos.lower,obitosAcumPreditos.upper,previsao
0,SP,Adamantina,350010,2020-03-27,-0.086165,4.284133,1,NaN,NaN,NaN,NaN,NaN
1,SP,Adamantina,350010,2020-03-28,-0.085077,4.198509,1,NaN,NaN,NaN,NaN,NaN
2,SP,Adamantina,350010,2020-03-29,-0.083951,4.113992,1,NaN,NaN,NaN,NaN,NaN
3,SP,Adamantina,350010,2020-03-30,-0.082792,4.030618,1,NaN,NaN,NaN,NaN,NaN
4,SP,Adamantina,350010,2020-03-31,-0.081603,3.948419,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
903821,SP,Valentim Gentil,355610,2021-12-12,0.006282,50.441929,0,0.0,0.049319,39.739294,63.725204,1.0
903822,SP,Valentim Gentil,355610,2021-12-13,0.000000,50.437480,0,0.0,0.001390,39.768938,63.740109,1.0
903823,SP,Valentim Gentil,355610,2021-12-14,0.000000,50.501785,0,0.0,0.000000,39.856219,63.801437,1.0
903824,SP,Valentim Gentil,355610,2021-12-15,0.000000,50.584832,0,0.0,0.000000,39.934604,63.887926,1.0


In [30]:
df_death_predictions

,estado,municipio,data,obitosPreditos,lower,upper,codmun,week_number_day
0,RO,Alta Floresta D'Oeste,2021-10-07,1,-0.453018,0.472176,110001,80.428571
1,RO,Alta Floresta D'Oeste,2021-10-08,1,-0.454492,0.471708,110001,80.571429
2,RO,Alta Floresta D'Oeste,2021-10-09,1,-0.455050,0.472266,110001,80.714286
3,RO,Alta Floresta D'Oeste,2021-10-10,1,-0.455608,0.472824,110001,80.857143
4,RO,Alta Floresta D'Oeste,2021-10-11,1,-0.456165,0.473381,110001,81.000000
...,...,...,...,...,...,...,...,...
167095,DF,Brasília,2021-11-01,12,-11.265490,31.493793,530010,84.000000
167096,DF,Brasília,2021-11-02,12,-11.643719,31.867547,530010,84.142857
167097,DF,Brasília,2021-11-03,12,-12.015981,32.238545,530010,84.285714
167098,DF,Brasília,2021-11-04,12,-12.381923,32.605302,530010,84.428571
